# Phase 2: 전처리 파이프라인

이 노트북에서 배울 내용:
- Voxel Grid 다운샘플링
- 노이즈 제거 (Statistical Outlier Removal)
- 법선 벡터 추정
- 전처리 파이프라인

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import open3d as o3d

from src.data import PointCloudLoader
from src.preprocessing import Preprocessor
from src.utils import Visualizer

## 1. 노이즈가 있는 샘플 데이터 생성

In [ ]:
loader = PointCloudLoader()
preprocessor = Preprocessor()

# 노이즈 포함 씬 생성
scene = loader.create_sample_scene(add_noise=True)
print(f"원본: {len(scene.points):,}개 포인트")

Visualizer.show(scene, "Original (with noise)")

## 2. Voxel Grid 다운샘플링

공간을 작은 큐브(voxel)로 나누고, 각 큐브 안의 점들을 하나의 대표점으로 대체합니다.

**장점**: 데이터 양 감소, 계산 속도 향상, 균일한 점 분포

**파라미터**: `voxel_size` - 클수록 점이 적어짐

In [ ]:
# 다양한 voxel size 비교
for voxel_size in [0.02, 0.05, 0.1, 0.2]:
    down = preprocessor.downsample(scene, voxel_size=voxel_size)
    print(f"voxel_size={voxel_size}: {len(down.points):,}개 포인트")

In [ ]:
# 시각적 비교
down_small = preprocessor.downsample(scene, voxel_size=0.03)
down_large = preprocessor.downsample(scene, voxel_size=0.1)

Visualizer.show_comparison(down_small, down_large, "voxel=0.03", "voxel=0.1")

## 3. 노이즈 제거

### Statistical Outlier Removal

각 점에 대해 k개의 이웃과의 평균 거리를 계산합니다.
전체 평균에서 표준편차의 `std_ratio`배 이상 벗어난 점을 제거합니다.

**파라미터**:
- `nb_neighbors`: 이웃 점 개수 (클수록 정교함)
- `std_ratio`: 허용 범위 (클수록 관대함)

In [ ]:
# 먼저 다운샘플링
scene_down = preprocessor.downsample(scene, voxel_size=0.05)

# 노이즈 제거
scene_clean, inlier_idx = preprocessor.remove_outliers(
    scene_down,
    nb_neighbors=20,
    std_ratio=2.0
)

In [ ]:
# 인라이어(정상)/아웃라이어(노이즈) 시각화
outlier_idx = np.setdiff1d(range(len(scene_down.points)), inlier_idx)

inlier_cloud = scene_down.select_by_index(inlier_idx)
outlier_cloud = scene_down.select_by_index(outlier_idx)
outlier_cloud.paint_uniform_color([1, 0, 0])  # 빨간색 = 제거됨

print(f"인라이어: {len(inlier_cloud.points):,}개")
print(f"아웃라이어: {len(outlier_cloud.points):,}개")

Visualizer.show_multiple([inlier_cloud, outlier_cloud], "Inliers (gray) + Outliers (red)")

## 4. 법선 벡터 추정

각 점의 국소 표면 방향을 나타내는 단위 벡터입니다.

**사용처**:
- Surface Reconstruction
- 특징 추출 (FPFH 등)
- 렌더링

In [ ]:
# 법선 추정
scene_with_normals = preprocessor.estimate_normals(scene_clean)

print(f"법선 벡터 수: {len(scene_with_normals.normals)}")

In [ ]:
# 법선 시각화 (선으로 표시)
# Open3D 뷰어에서 'N' 키를 누르면 법선 표시 토글
o3d.visualization.draw_geometries(
    [scene_with_normals],
    window_name="Normals (press N to toggle)",
    point_show_normal=True
)

## 5. 전체 파이프라인

In [ ]:
# 원본 데이터 다시 로드
scene_original = loader.create_sample_scene(add_noise=True)

# 전체 파이프라인 실행
scene_processed = preprocessor.full_pipeline(
    scene_original,
    voxel_size=0.05,
    remove_noise=True,
    estimate_normals=True
)

In [ ]:
# 전후 비교
print(f"원본: {len(scene_original.points):,}개")
print(f"처리 후: {len(scene_processed.points):,}개")

Visualizer.show_comparison(scene_original, scene_processed, "원본", "전처리 후")

## 6. 파라미터 튜닝 가이드

| 파라미터 | 작게 | 크게 |
|---------|------|------|
| voxel_size | 점 많음, 디테일 유지 | 점 적음, 빠른 처리 |
| nb_neighbors | 빠름, 덜 정교 | 느림, 더 정교 |
| std_ratio | 공격적 제거 | 관대한 제거 |

In [ ]:
# 파라미터별 효과 실험
# TODO: 다양한 파라미터를 시험해보세요


## 연습 문제

1. `std_ratio`를 1.0, 2.0, 3.0으로 변경하며 제거되는 점의 수를 비교해보세요.
2. 노이즈가 매우 심한 데이터를 생성하고 효과적인 파라미터를 찾아보세요.
3. 전처리 전후 Point Cloud를 파일로 저장해보세요.

In [ ]:
# 연습 코드 작성
